# Banda derecha

Vamos a seleccionar al mejor interior derecho según:

    1. Porcentaje de pases acertados
    2. Asistencias
    3. Porcentaje de regates acertados 
    4. Goles

In [1]:
import pandas as pd
import numpy as np

#leemos el csv y eliminamos duplicidades
df = pd.read_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/raw/datasetMatchInfo2020-La-Liga.csv')
df = df.drop_duplicates()



In [2]:
'''Creamos un array donde aparezcan los nombres de todos los jugadores que, 
según el dataset, al menos una vez han jugado como interior derecho'''
df_rm = df[df['Pos'] == 'RM']
df_rm_unique = df_rm['Player'].unique()


In [3]:
'''
Creamos un bucle for que lea todos los nombres del array para 
que les asigne la posición RM en el dataframe en todos los partidos.
'''

for i in df_rm_unique:
    if np.any(df['Player'] == i):
        df.loc[(df['Player']== i), 'Pos'] = 'RM'



/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83589/3826672003.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'RM'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83589/3826672003.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'RM'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83589/3826672003.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [4]:
'''
Para segmentar, establecemos que queremos obtener únicamente los partidos
en los que el futbolista haya jugado más de 45 minutos
'''
df_rm = df[df['Pos'] == 'RM']
df_rm =df_rm[df_rm['Min']>= 45]


In [5]:
df_rm.groupby('Player')['Performance|Touches'].mean().mean()
#la media de las medias de toques de balón por partido de los interiores derechos es de 40 toques

39.23699078699079

In [6]:
#Creamos un dataframe con los minutos jugados por jugador
#Segmentamos por los jugadores que hayan jugado, al menos, un total de 500 minutos.

df_minutos = df_rm.groupby('Player')['Min'].sum()
df_minutos_rm = df_minutos.loc[df_minutos>500].sort_values(ascending =False)


In [7]:
#Creamos el nuevo dataframe con los futbolistas que han jugado más de 500 minutos
df_rm= df_rm[df_rm['Player'].isin(df_minutos_rm.keys())]


In [8]:
#como el mínimo de minutos es de 720 (más de 8 partidos), establezco un mínimo
#de 300 toques de balón en total.
df_toques= df_rm.groupby('Player')['Performance|Touches'].sum()
df_toques_rm = df_toques.loc[df_toques>= (300)].sort_values(ascending=False)


In [9]:
#aplico el filtro al dataframe
df_rm= df_rm[df_rm['Player'].isin(df_toques_rm.keys())]


In [10]:
'''
Creamos un dataframe con la media de recuperaciones de balón por partido.
Posteriormente los ponderamos entre el total.
'''
df_recov_try= df_rm.groupby('Player')['Performance|Recov'].mean().sort_values(ascending=False)
df_recov = df_recov_try[df_recov_try.keys().isin(df_minutos_rm.keys())]
df_recov = df_recov/df_recov.sum()


In [11]:
'''
Creamos un dataframe con las asistencias realizadas por jugador.
'''
df_ast_try= df_rm.groupby('Player')['Performance|Ast'].sum().sort_values(ascending=False)
df_ast = df_ast_try[df_ast_try.keys().isin(df_minutos_rm.keys())]


In [12]:
'''
Creamos un dataframe con la media de los porcentajes de pases acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_cmp_try= df_rm.groupby('Player')['Passes|Cmp%'].mean().sort_values(ascending=False)
df_cmp= df_cmp_try[df_cmp_try.keys().isin(df_minutos_rm.keys())]
df_cmp= df_cmp/df_cmp.sum()



In [13]:
'''
Creamos un dataframe con los goles anotados por jugador.
'''
df_gls_try= df_rm.groupby('Player')['Performance|Gls'].sum().sort_values(ascending=False)
df_gls = df_gls_try[df_gls_try.keys().isin(df_minutos_rm.keys())]
df_gls = df_gls[2:] #para quitar a Gerard Moreno y a Iago Aspas, dos delanteros que aparecen como bandas


In [14]:
'''
Creamos un dataframe con la media de los porcentajes de regates acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_drbls_try= df_rm.groupby('Player')['Dribbles|Succ%'].sum().sort_values(ascending=False)
df_drbls_succ = df_drbls_try[df_drbls_try.keys().isin(df_minutos_rm.keys())]
df_drbls_succ = df_drbls_succ/(df_drbls_succ.sum())


In [15]:
#Convertimos las series a dataframe
df_drbls_succ = df_drbls_succ.to_frame()
df_recov=df_recov.to_frame()
df_cmp = df_cmp.to_frame()
df_ast = df_ast.to_frame()
df_gls = df_gls.to_frame()



In [16]:
'''
Unimos todos los dataframes por la columna Player
'''
df_rm_final = df_cmp.merge(df_ast, how='inner', on=['Player'])
df_rm_final = df_rm_final.merge(df_gls, how='inner', on=['Player'])
df_rm_final =df_rm_final.merge(df_drbls_succ, how='inner', on=['Player'])


In [17]:
#añadimos la columna con los equipos de cada jugador al dataframe final
df_teams= df[['Player','Team']]
df_rm_final = df_teams.merge(df_rm_final, how='inner', on='Player').drop_duplicates()


In [18]:
#Establecemos la columna de Puntuación final
df_rm_final['Puntuación Final'] = (df_rm_final.iloc[:,2]) + (df_rm_final.iloc[:,3]*0.02) + (df_rm_final.iloc[:,4]*0.02) + (df_rm_final.iloc[:,5])
df_rm = df_rm_final.sort_values(by='Puntuación Final', ascending=False)



In [19]:
#Cambiamos los nombres de las columnas
#Pongo iloc[3:] porque realmente ni Joao Felix, ni Oyarzabal ni Correa son interiores derecho.
df_rm.columns =['Jugador', 'Equipo', 'Porcentaje de pases acertados', 'Asistencias', 'Goles', 'Porcentaje de regates acertados', 'Puntuación Final']
df_rm = df_rm.sort_values(by= 'Puntuación Final', ascending = False).iloc[3:]
df_rm


,Jugador,Equipo,Porcentaje de pases acertados,Asistencias,Goles,Porcentaje de regates acertados,Puntuación Final
255,Marcos Llorente,Atlético,0.041424,1.0,4.0,0.028092,0.169516
183,Rubén García,Osasuna,0.032798,2.0,3.0,0.020457,0.153255
32,Manu Trigueros,Villarreal,0.042869,1.0,1.0,0.053189,0.136058
23,Cucho,Getafe,0.034579,2.0,1.0,0.038460,0.133039
89,Josan,Elche,0.038730,1.0,1.0,0.047052,0.125781
146,David Ferreiro,Huesca,0.030430,2.0,1.0,0.034098,0.124529
159,Denis Suárez,Celta,0.039434,2.0,0.0,0.043639,0.123073
111,Pedri,Barcelona,0.042850,1.0,1.0,0.036823,0.119673
100,Fabián Orellana,Valladolid,0.037018,0.0,2.0,0.039548,0.116566
0,Joaquín,Betis,0.040341,2.0,0.0,0.036005,0.116346


In [20]:
#Exportar a csv
df_rm.to_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/processed/csv_principales/7_Banda_Derecha.csv')